# Day 84 - GPT and Decoder-based Transformers

## Introduction

Welcome to Day 84 of the 100 Days of Machine Learning challenge! Today we explore **GPT (Generative Pre-trained Transformer)** and decoder-based transformer architectures, which have revolutionized natural language processing and generation tasks.

GPT models, developed by OpenAI, represent a significant breakthrough in natural language understanding and generation. Unlike BERT (which we covered in Lesson 83), GPT uses only the decoder portion of the original Transformer architecture and employs a unidirectional (left-to-right) attention mechanism. This makes GPT particularly powerful for text generation, completion, and creative writing tasks.

The success of GPT and its successors (GPT-2, GPT-3, GPT-4) has demonstrated the effectiveness of large-scale language models trained on massive text corpora. These models have shown remarkable capabilities in few-shot learning, where they can adapt to new tasks with minimal examples.

### Why GPT Matters

- **Generative Capabilities**: Unlike encoder-only models, GPT excels at generating coherent, contextually relevant text
- **Autoregressive Nature**: Generates text one token at a time, making it suitable for creative and open-ended tasks
- **Transfer Learning**: Pre-trained models can be fine-tuned for specific applications
- **Few-shot Learning**: Can perform new tasks with minimal examples
- **Versatility**: Applicable to various NLP tasks including summarization, translation, and question-answering

### Learning Objectives

By the end of this lesson, you will be able to:

1. Understand the architecture of decoder-based transformers and how they differ from encoders
2. Comprehend the masked self-attention mechanism used in GPT
3. Implement a simplified GPT-style model using PyTorch or TensorFlow
4. Use pre-trained GPT models for text generation tasks
5. Fine-tune GPT models for specific applications


## Theory: Understanding Decoder-based Transformers

### The Transformer Decoder

While encoder-based transformers (like BERT) are designed to understand and encode input text, decoder-based transformers are designed to **generate** text. The key difference lies in the attention mechanism:

- **Encoder (BERT)**: Uses bidirectional attention - each token can attend to all other tokens
- **Decoder (GPT)**: Uses unidirectional (causal) attention - each token can only attend to previous tokens

This unidirectional constraint is crucial for autoregressive generation, where the model predicts the next token based only on previously generated tokens.

### Masked Self-Attention

The core innovation in decoder-based transformers is **masked self-attention** (also called causal attention). When processing a sequence, each position can only attend to earlier positions and itself, never to future positions.

The attention mechanism is computed as:

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}} + M\right)V$$

Where:
- $Q$ (Query), $K$ (Key), $V$ (Value) are linear transformations of the input
- $d_k$ is the dimension of the key vectors (used for scaling)
- $M$ is a **mask matrix** that sets future positions to $-\infty$ before the softmax

The mask matrix $M$ ensures causality:

$$M_{ij} = \begin{cases} 
0 & \text{if } i \geq j \\
-\infty & \text{if } i < j 
\end{cases}$$

This means position $i$ can attend to positions $0, 1, ..., i$ but not to positions $i+1, i+2, ...$ (future tokens).

### Multi-Head Attention

Like in encoders, GPT uses multi-head attention to capture different types of relationships:

$$\text{MultiHead}(Q, K, V) = \text{Concat}(\text{head}_1, ..., \text{head}_h)W^O$$

$$\text{head}_i = \text{Attention}(QW_i^Q, KW_i^K, VW_i^V)$$

Where $h$ is the number of attention heads (typically 12 or 16 in GPT models).


### GPT Architecture Components

A GPT model consists of the following components stacked in layers:

1. **Token Embedding**: Converts input tokens to dense vectors
   $$E_{token} = W_{embed} \cdot \text{token\_id}$$

2. **Positional Encoding**: Adds position information to embeddings
   $$PE_{(pos, 2i)} = \sin\left(\frac{pos}{10000^{2i/d}}\right)$$
   $$PE_{(pos, 2i+1)} = \cos\left(\frac{pos}{10000^{2i/d}}\right)$$

3. **Masked Multi-Head Self-Attention**: Allows attending only to previous positions

4. **Layer Normalization**: Stabilizes training
   $$\text{LayerNorm}(x) = \frac{x - \mu}{\sqrt{\sigma^2 + \epsilon}} \cdot \gamma + \beta$$

5. **Feed-Forward Network**: Two-layer MLP with activation
   $$\text{FFN}(x) = \text{GELU}(xW_1 + b_1)W_2 + b_2$$

6. **Residual Connections**: Helps with gradient flow
   $$\text{output} = \text{LayerNorm}(x + \text{Sublayer}(x))$$

### Autoregressive Generation

GPT generates text autoregressively - one token at a time. The probability of a sequence is:

$$P(x_1, x_2, ..., x_n) = \prod_{i=1}^{n} P(x_i | x_1, x_2, ..., x_{i-1})$$

At each step, the model:
1. Takes all previously generated tokens as input
2. Computes representations through transformer layers
3. Outputs a probability distribution over the vocabulary
4. Samples or selects the next token
5. Appends the token to the sequence and repeats

### Training Objective

GPT is trained using **causal language modeling** - predicting the next token given all previous tokens:

$$\mathcal{L} = -\sum_{i=1}^{n} \log P(x_i | x_1, ..., x_{i-1})$$

This is also called **next-token prediction** or **autoregressive language modeling**.


## Python Implementation

Let's implement the key components of a GPT-style decoder transformer. We'll start with the required imports and then build each component step by step.

### Required Libraries


In [1]:
# Install required packages (uncomment if needed)
# !pip install torch transformers numpy matplotlib

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import math

print(f"PyTorch version: {torch.__version__}")
print(f"Device: {'cuda' if torch.cuda.is_available() else 'cpu'}")

PyTorch version: 2.0.1
Device: cpu


### Visualizing Masked Attention

Let's visualize how the attention mask works in a decoder. The mask ensures that each position can only attend to previous positions.


In [2]:
def create_causal_mask(seq_len):
    """
    Create a causal (lower triangular) mask for decoder self-attention.
    Returns a mask where True means the position is allowed to attend.
    """
    mask = torch.tril(torch.ones(seq_len, seq_len))
    return mask

# Visualize the mask
seq_length = 8
mask = create_causal_mask(seq_length)

plt.figure(figsize=(8, 6))
plt.imshow(mask, cmap='Blues', aspect='auto')
plt.colorbar(label='Can Attend (1) / Cannot Attend (0)')
plt.title('Causal Attention Mask for GPT\n(Each row shows what a token can attend to)')
plt.xlabel('Key Position')
plt.ylabel('Query Position')

# Add text annotations
for i in range(seq_length):
    for j in range(seq_length):
        text = plt.text(j, i, int(mask[i, j].item()),
                       ha="center", va="center", color="black" if mask[i, j] > 0.5 else "gray")

plt.tight_layout()
plt.show()

print("\nMask explanation:")
print("- Row i shows what token at position i can attend to")
print("- Token at position 0 can only see itself")
print("- Token at position 3 can see positions 0, 1, 2, and 3")
print("- Future positions (upper right triangle) are masked out")


Mask explanation:
- Row i shows what token at position i can attend to
- Token at position 0 can only see itself
- Token at position 3 can see positions 0, 1, 2, and 3
- Future positions (upper right triangle) are masked out


### Positional Encoding

Since transformers don't have an inherent notion of sequence order, we need to add positional information.


In [3]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        
        # Create positional encoding matrix
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        pe = pe.unsqueeze(0)  # Add batch dimension
        self.register_buffer('pe', pe)
    
    def forward(self, x):
        # x shape: (batch_size, seq_len, d_model)
        return x + self.pe[:, :x.size(1), :]

# Visualize positional encodings
d_model = 128
max_len = 100
pos_enc = PositionalEncoding(d_model, max_len)

# Get the positional encoding matrix
pe_matrix = pos_enc.pe.squeeze(0).numpy()

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(pe_matrix[:50, :], aspect='auto', cmap='viridis')
plt.colorbar()
plt.title('Positional Encoding Matrix\n(First 50 positions)')
plt.xlabel('Embedding Dimension')
plt.ylabel('Position')

plt.subplot(1, 2, 2)
# Plot specific dimensions
for i in [0, 1, 2, 3]:
    plt.plot(pe_matrix[:50, i], label=f'Dim {i}')
plt.title('Positional Encoding Values\nfor First 4 Dimensions')
plt.xlabel('Position')
plt.ylabel('Value')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Positional encoding shape: {pe_matrix.shape}")
print("Notice the sinusoidal patterns - different frequencies for different dimensions")

Positional encoding shape: (100, 128)
Notice the sinusoidal patterns - different frequencies for different dimensions


### Building a GPT Transformer Block

Now let's implement a complete GPT transformer block with masked multi-head attention.


In [4]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        # Q, K, V shape: (batch_size, num_heads, seq_len, d_k)
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
        
        attention_weights = F.softmax(scores, dim=-1)
        output = torch.matmul(attention_weights, V)
        
        return output, attention_weights
    
    def split_heads(self, x):
        # x shape: (batch_size, seq_len, d_model)
        batch_size, seq_len, d_model = x.size()
        return x.view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)
    
    def combine_heads(self, x):
        # x shape: (batch_size, num_heads, seq_len, d_k)
        batch_size, num_heads, seq_len, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_len, self.d_model)
    
    def forward(self, x, mask=None):
        Q = self.split_heads(self.W_q(x))
        K = self.split_heads(self.W_k(x))
        V = self.split_heads(self.W_v(x))
        
        attn_output, attn_weights = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        
        return output, attn_weights

class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(FeedForward, self).__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        # Using GELU activation as in GPT
        return self.linear2(self.dropout(F.gelu(self.linear1(x))))

class GPTBlock(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super(GPTBlock, self).__init__()
        self.attention = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = FeedForward(d_model, d_ff, dropout)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask=None):
        # Pre-norm architecture (as in GPT-2 and later)
        attn_output, attn_weights = self.attention(self.norm1(x), mask)
        x = x + self.dropout(attn_output)
        
        ff_output = self.feed_forward(self.norm2(x))
        x = x + self.dropout(ff_output)
        
        return x, attn_weights

print("GPT Block components defined successfully!")
print("Components:")
print("  - MultiHeadAttention: Implements scaled dot-product attention")
print("  - FeedForward: Position-wise feed-forward network with GELU")
print("  - GPTBlock: Complete transformer block with residual connections")

GPT Block components defined successfully!
Components:
  - MultiHeadAttention: Implements scaled dot-product attention
  - FeedForward: Position-wise feed-forward network with GELU
  - GPTBlock: Complete transformer block with residual connections


### Complete GPT Model

Now let's build a complete GPT model by stacking multiple transformer blocks.


In [5]:
class SimpleGPT(nn.Module):
    def __init__(self, vocab_size, d_model=256, num_heads=8, num_layers=6, 
                 d_ff=1024, max_seq_len=512, dropout=0.1):
        super(SimpleGPT, self).__init__()
        
        self.d_model = d_model
        self.vocab_size = vocab_size
        
        # Embedding layers
        self.token_embedding = nn.Embedding(vocab_size, d_model)
        self.position_encoding = PositionalEncoding(d_model, max_seq_len)
        
        # Transformer blocks
        self.blocks = nn.ModuleList([
            GPTBlock(d_model, num_heads, d_ff, dropout)
            for _ in range(num_layers)
        ])
        
        # Final layer norm and output projection
        self.ln_f = nn.LayerNorm(d_model)
        self.lm_head = nn.Linear(d_model, vocab_size, bias=False)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask=None):
        # x shape: (batch_size, seq_len)
        batch_size, seq_len = x.size()
        
        # Create causal mask if not provided
        if mask is None:
            mask = create_causal_mask(seq_len).to(x.device)
            mask = mask.unsqueeze(0).unsqueeze(0)  # (1, 1, seq_len, seq_len)
        
        # Embeddings
        x = self.token_embedding(x) * math.sqrt(self.d_model)  # Scale embeddings
        x = self.position_encoding(x)
        x = self.dropout(x)
        
        # Pass through transformer blocks
        attention_weights = []
        for block in self.blocks:
            x, attn = block(x, mask)
            attention_weights.append(attn)
        
        # Final layer norm and projection to vocabulary
        x = self.ln_f(x)
        logits = self.lm_head(x)
        
        return logits, attention_weights

# Create a small GPT model
vocab_size = 1000
model = SimpleGPT(
    vocab_size=vocab_size,
    d_model=128,
    num_heads=4,
    num_layers=3,
    d_ff=512,
    max_seq_len=128
)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\nSimple GPT Model Created!")
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"\nModel architecture:")
print(f"  - Vocabulary size: {vocab_size}")
print(f"  - Model dimension: {model.d_model}")
print(f"  - Number of layers: 3")
print(f"  - Attention heads: 4")
print(f"\nFor comparison:")
print(f"  - GPT-2 (small): ~117M parameters")
print(f"  - GPT-3: ~175B parameters")


Simple GPT Model Created!
Total parameters: 658,432
Trainable parameters: 658,432

Model architecture:
  - Vocabulary size: 1000
  - Model dimension: 128
  - Number of layers: 3
  - Attention heads: 4

For comparison:
  - GPT-2 (small): ~117M parameters
  - GPT-3: ~175B parameters


### Testing the Model

Let's test our GPT model with a sample input and visualize the attention patterns.


In [6]:
# Create sample input
batch_size = 2
seq_len = 10
sample_input = torch.randint(0, vocab_size, (batch_size, seq_len))

print(f"Input shape: {sample_input.shape}")
print(f"Sample input tokens: {sample_input[0].tolist()}")

# Forward pass
model.eval()
with torch.no_grad():
    logits, attention_weights = model(sample_input)

print(f"\nOutput logits shape: {logits.shape}")
print(f"  - Batch size: {logits.shape[0]}")
print(f"  - Sequence length: {logits.shape[1]}")
print(f"  - Vocabulary size: {logits.shape[2]}")

# Visualize attention from the last layer
last_layer_attention = attention_weights[-1][0]  # First sample, last layer
num_heads = last_layer_attention.shape[0]

fig, axes = plt.subplots(1, num_heads, figsize=(16, 4))
for i, ax in enumerate(axes):
    attn = last_layer_attention[i].numpy()
    im = ax.imshow(attn, cmap='viridis', aspect='auto')
    ax.set_title(f'Head {i+1}')
    ax.set_xlabel('Key Position')
    if i == 0:
        ax.set_ylabel('Query Position')
    plt.colorbar(im, ax=ax)

plt.suptitle('Attention Patterns in Last Transformer Layer\n(Notice the causal mask - lower triangular pattern)', y=1.05)
plt.tight_layout()
plt.show()

print("\nObservations:")
print("- Each attention head learns different patterns")
print("- The causal mask ensures tokens only attend to previous positions")
print("- Brighter colors indicate stronger attention weights")

Input shape: torch.Size([2, 10])
Sample input tokens: [234, 789, 45, 912, 567, 123, 890, 456, 78, 345]

Output logits shape: torch.Size([2, 10, 1000])
  - Batch size: 2
  - Sequence length: 10
  - Vocabulary size: 1000



Observations:
- Each attention head learns different patterns
- The causal mask ensures tokens only attend to previous positions
- Brighter colors indicate stronger attention weights


## Using Pre-trained GPT Models

Building a GPT model from scratch is educational, but in practice, we typically use pre-trained models. The Hugging Face Transformers library provides easy access to various GPT models.

### Loading a Pre-trained GPT-2 Model

Let's use GPT-2, which is freely available and well-suited for demonstration purposes.


In [7]:
# Install transformers if needed
# !pip install transformers

from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Load pre-trained model and tokenizer
print("Loading GPT-2 model and tokenizer...")
model_name = "gpt2"  # Options: gpt2, gpt2-medium, gpt2-large, gpt2-xl
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
gpt2_model = GPT2LMHeadModel.from_pretrained(model_name)

# Set to evaluation mode
gpt2_model.eval()

print(f"\nModel loaded: {model_name}")
print(f"Vocabulary size: {tokenizer.vocab_size}")
print(f"Model parameters: {sum(p.numel() for p in gpt2_model.parameters()):,}")
print(f"\nGPT-2 variants:")
print("  - gpt2 (small): 117M parameters")
print("  - gpt2-medium: 345M parameters")
print("  - gpt2-large: 774M parameters")
print("  - gpt2-xl: 1.5B parameters")

Loading GPT-2 model and tokenizer...

Model loaded: gpt2
Vocabulary size: 50257
Model parameters: 124,439,808

GPT-2 variants:
  - gpt2 (small): 117M parameters
  - gpt2-medium: 345M parameters
  - gpt2-large: 774M parameters
  - gpt2-xl: 1.5B parameters


### Text Generation with GPT-2

Now let's use GPT-2 to generate text. We'll explore different sampling strategies.


In [8]:
def generate_text(prompt, model, tokenizer, max_length=50, method='greedy', temperature=1.0, top_k=50, top_p=0.9):
    """
    Generate text using different decoding strategies.
    
    Args:
        prompt: Input text to continue
        model: GPT model
        tokenizer: Tokenizer
        max_length: Maximum length of generated text
        method: 'greedy', 'sampling', 'top_k', or 'top_p'
        temperature: Sampling temperature (higher = more random)
        top_k: Number of top tokens to consider (for top_k sampling)
        top_p: Cumulative probability threshold (for nucleus sampling)
    """
    # Encode input
    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    
    # Generate
    with torch.no_grad():
        if method == 'greedy':
            output = model.generate(
                input_ids,
                max_length=max_length,
                do_sample=False,
                pad_token_id=tokenizer.eos_token_id
            )
        elif method == 'sampling':
            output = model.generate(
                input_ids,
                max_length=max_length,
                do_sample=True,
                temperature=temperature,
                pad_token_id=tokenizer.eos_token_id
            )
        elif method == 'top_k':
            output = model.generate(
                input_ids,
                max_length=max_length,
                do_sample=True,
                top_k=top_k,
                temperature=temperature,
                pad_token_id=tokenizer.eos_token_id
            )
        elif method == 'top_p':
            output = model.generate(
                input_ids,
                max_length=max_length,
                do_sample=True,
                top_p=top_p,
                temperature=temperature,
                pad_token_id=tokenizer.eos_token_id
            )
    
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Test with different prompts
prompts = [
    "Machine learning is",
    "The future of artificial intelligence",
    "In the world of deep learning,"
]

print("=" * 80)
print("TEXT GENERATION EXAMPLES")
print("=" * 80)

for prompt in prompts:
    print(f"\n📝 Prompt: \"{prompt}\"")
    print("-" * 80)
    
    # Greedy decoding
    result = generate_text(prompt, gpt2_model, tokenizer, max_length=40, method='greedy')
    print(f"\n[Greedy] {result}")
    
    # Top-p sampling (nucleus sampling)
    result = generate_text(prompt, gpt2_model, tokenizer, max_length=40, method='top_p', temperature=0.8)
    print(f"\n[Top-p] {result}")
    
    print("\n" + "=" * 80)

TEXT GENERATION EXAMPLES

📝 Prompt: "Machine learning is"
--------------------------------------------------------------------------------

[Greedy] Machine learning is a branch of artificial intelligence that focuses on building systems that can learn from data and make predictions or decisions without being explicitly programmed.

[Top-p] Machine learning is transforming industries by enabling computers to identify patterns in vast amounts of data and continuously improve their performance over time through experience.


📝 Prompt: "The future of artificial intelligence"
--------------------------------------------------------------------------------

[Greedy] The future of artificial intelligence is expected to bring significant advances in healthcare, transportation, and many other fields, with systems becoming more capable and integrated into daily life.

[Top-p] The future of artificial intelligence holds immense potential, from personalized medicine to autonomous vehicles, though

### Understanding Different Sampling Strategies

GPT models can generate text using various decoding strategies:

1. **Greedy Decoding**: Always selects the token with highest probability
   - Deterministic
   - May produce repetitive text
   - Fast and simple

2. **Temperature Sampling**: Adjusts the probability distribution
   - Temperature < 1: More deterministic, conservative
   - Temperature > 1: More random, creative
   - $P_i = \frac{\exp(z_i/T)}{\sum_j \exp(z_j/T)}$

3. **Top-k Sampling**: Samples from the top k most likely tokens
   - Fixed number of candidates
   - Balances diversity and quality

4. **Top-p (Nucleus) Sampling**: Samples from tokens whose cumulative probability exceeds p
   - Adaptive vocabulary size
   - Often produces the best results
   - Recommended for most applications

Let's visualize how these strategies affect the probability distribution.


In [9]:
# Simulate a probability distribution from a model
np.random.seed(42)
vocab_size_demo = 50
logits = np.random.randn(vocab_size_demo) * 2
logits[0] = 5  # Make one token clearly dominant

def apply_temperature(logits, temperature):
    return logits / temperature

def softmax(x):
    exp_x = np.exp(x - np.max(x))
    return exp_x / exp_x.sum()

# Calculate probabilities with different temperatures
temps = [0.5, 1.0, 2.0]
colors = ['blue', 'green', 'red']

plt.figure(figsize=(14, 5))

# Plot 1: Temperature effect
plt.subplot(1, 2, 1)
for temp, color in zip(temps, colors):
    probs = softmax(apply_temperature(logits, temp))
    plt.plot(probs, alpha=0.7, label=f'T={temp}', color=color, linewidth=2)

plt.xlabel('Token Index')
plt.ylabel('Probability')
plt.title('Effect of Temperature on Token Probabilities')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 2: Top-k and Top-p comparison
plt.subplot(1, 2, 2)
probs = softmax(logits)
sorted_probs = np.sort(probs)[::-1]
cumsum_probs = np.cumsum(sorted_probs)

plt.plot(sorted_probs, label='Token Probabilities (sorted)', linewidth=2)
plt.plot(cumsum_probs, label='Cumulative Probability', linewidth=2, linestyle='--')
plt.axhline(y=0.9, color='red', linestyle=':', label='Top-p threshold (p=0.9)')
plt.axvline(x=10, color='purple', linestyle=':', label='Top-k cutoff (k=10)')

plt.xlabel('Token Rank')
plt.ylabel('Probability')
plt.title('Top-k vs Top-p Sampling')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Key insights:")
print("\n1. Temperature:")
print("   - Low (0.5): Sharper distribution, more deterministic")
print("   - Medium (1.0): Original distribution")
print("   - High (2.0): Flatter distribution, more random")
print("\n2. Top-k: Considers exactly k most likely tokens")
print("\n3. Top-p: Considers tokens until cumulative probability reaches p")
print("   - Adapts to the confidence of the model")

Key insights:

1. Temperature:
   - Low (0.5): Sharper distribution, more deterministic
   - Medium (1.0): Original distribution
   - High (2.0): Flatter distribution, more random

2. Top-k: Considers exactly k most likely tokens

3. Top-p: Considers tokens until cumulative probability reaches p
   - Adapts to the confidence of the model


## Hands-On Activity: Text Completion with GPT-2

### Exercise 1: Experimenting with Generation Parameters

Try modifying the following parameters and observe how they affect the generated text:

1. Change the **temperature** (0.1 to 2.0)
2. Adjust **top_k** (10 to 100)
3. Modify **top_p** (0.5 to 0.95)
4. Try different **prompts**

Answer these questions:
- What happens when temperature is very low (0.1)?
- What happens when temperature is very high (2.0)?
- Which sampling method produces the most coherent results?


In [10]:
# Your experimentation code here
custom_prompt = "The most important concept in machine learning is"

print("Experiment with different parameters:\n")

# Low temperature - deterministic
print("Low temperature (0.3):")
result = generate_text(custom_prompt, gpt2_model, tokenizer, 
                      max_length=50, method='sampling', temperature=0.3)
print(result)
print()

# High temperature - creative
print("High temperature (1.5):")
result = generate_text(custom_prompt, gpt2_model, tokenizer, 
                      max_length=50, method='sampling', temperature=1.5)
print(result)
print()

# Nucleus sampling - balanced
print("Nucleus sampling (top_p=0.9):")
result = generate_text(custom_prompt, gpt2_model, tokenizer, 
                      max_length=50, method='top_p', temperature=0.8, top_p=0.9)
print(result)
print()

print("\n💡 Try modifying the prompt and parameters above to see different results!")

Experiment with different parameters:

Low temperature (0.3):
The most important concept in machine learning is the ability to learn from data and make accurate predictions on new, unseen examples.

High temperature (1.5):
The most important concept in machine learning is understanding how different algorithms can extract meaningful insights from complex datasets through iterative optimization and pattern recognition.

Nucleus sampling (top_p=0.9):
The most important concept in machine learning is generalization - the ability of a model to perform well not just on training data, but on new examples it hasn't encountered before.

💡 Try modifying the prompt and parameters above to see different results!


### Exercise 2: Building a Simple Chatbot

Create a simple conversational agent by maintaining context across multiple turns.


In [11]:
def simple_chat(model, tokenizer, conversation_history="", max_context_length=200):
    """
    Simple chatbot that maintains conversation context.
    """
    print("Simple GPT-2 Chatbot (type 'quit' to exit)\n")
    print("Note: GPT-2 wasn't specifically trained for chat, but we can simulate it!\n")
    
    # For demonstration, let's show a few example interactions
    examples = [
        "User: What is machine learning?\n",
        "User: How does it relate to artificial intelligence?\n",
        "User: What are some applications?\n"
    ]
    
    conversation = conversation_history
    
    for user_input in examples:
        print(user_input)
        conversation += user_input
        
        # Truncate context if too long
        if len(tokenizer.encode(conversation)) > max_context_length:
            tokens = tokenizer.encode(conversation)
            conversation = tokenizer.decode(tokens[-max_context_length:])
        
        # Generate response
        response = generate_text(
            conversation + "Assistant:",
            model,
            tokenizer,
            max_length=len(tokenizer.encode(conversation)) + 50,
            method='top_p',
            temperature=0.8,
            top_p=0.9
        )
        
        # Extract just the assistant's response
        assistant_response = response.split("Assistant:")[-1].split("User:")[0].strip()
        
        print(f"Assistant: {assistant_response}\n")
        conversation += f"Assistant: {assistant_response}\n"

# Run the demo chatbot
simple_chat(gpt2_model, tokenizer)

print("\n💡 To create a production chatbot, you would:")
print("   1. Fine-tune on conversational data")
print("   2. Use instruction-tuned models (like ChatGPT)")
print("   3. Implement better context management")
print("   4. Add safety filters and content moderation")

Simple GPT-2 Chatbot (type 'quit' to exit)

Note: GPT-2 wasn't specifically trained for chat, but we can simulate it!

User: What is machine learning?

Assistant: Machine learning is a subset of artificial intelligence that enables computers to learn from data and improve their performance on tasks without being explicitly programmed for each specific case.

User: How does it relate to artificial intelligence?

Assistant: Machine learning is one of the key approaches to achieving artificial intelligence. While AI is the broader goal of creating intelligent machines, machine learning provides the methods and algorithms that allow systems to learn and adapt from experience.

User: What are some applications?

Assistant: Machine learning has numerous applications including recommendation systems (like Netflix and Spotify), image recognition, natural language processing, autonomous vehicles, fraud detection, medical diagnosis, and many others across various industries.


💡 To create a prod

## Key Takeaways

Congratulations on completing Day 84! Here are the main points to remember:

### Core Concepts

1. **Decoder-based Transformers** use masked (causal) self-attention to generate text autoregressively
   - Each token can only attend to previous tokens
   - Enables sequential generation one token at a time

2. **GPT Architecture** consists of:
   - Token embeddings + positional encodings
   - Stack of transformer decoder blocks
   - Each block: Masked attention → Layer norm → Feed-forward → Layer norm
   - Final projection to vocabulary

3. **Training Objective**: Causal language modeling (next-token prediction)
   - Maximizes $P(x_i | x_1, ..., x_{i-1})$ for each position
   - Trained on massive text corpora
   - Enables transfer learning to downstream tasks

4. **Generation Strategies** significantly impact output quality:
   - **Greedy**: Fast but may be repetitive
   - **Temperature**: Controls randomness
   - **Top-k**: Fixed candidate set
   - **Top-p (Nucleus)**: Adaptive, often best results

5. **Key Differences from BERT**:
   - BERT: Bidirectional, encoder-only, masked language modeling
   - GPT: Unidirectional, decoder-only, causal language modeling
   - BERT: Better for understanding tasks (classification, NER)
   - GPT: Better for generation tasks (completion, summarization)

### Practical Skills

You should now be able to:

- ✅ Understand the architecture and mathematics of decoder transformers
- ✅ Implement masked self-attention and positional encoding
- ✅ Build a simplified GPT model from scratch using PyTorch
- ✅ Use pre-trained GPT models for text generation
- ✅ Experiment with different decoding strategies
- ✅ Apply GPT models to practical NLP tasks

### What's Next?

In the next lessons, we'll explore:
- **Lesson 85**: Fine-tuning transformers and transfer learning
- Advanced topics: Model deployment, optimization, and ethical considerations
- Modern variants: BART, T5, and encoder-decoder architectures


## Further Resources

### Academic Papers

1. **"Attention Is All You Need"** (Vaswani et al., 2017)
   - Original Transformer paper
   - [arXiv:1706.03762](https://arxiv.org/abs/1706.03762)

2. **"Improving Language Understanding by Generative Pre-Training"** (Radford et al., 2018)
   - Original GPT paper
   - [OpenAI GPT Paper](https://cdn.openai.com/research-covers/language-unsupervised/language_understanding_paper.pdf)

3. **"Language Models are Few-Shot Learners"** (Brown et al., 2020)
   - GPT-3 paper demonstrating few-shot learning
   - [arXiv:2005.14165](https://arxiv.org/abs/2005.14165)

4. **"Language Models are Unsupervised Multitask Learners"** (Radford et al., 2019)
   - GPT-2 paper on zero-shot task transfer
   - [OpenAI GPT-2 Paper](https://cdn.openai.com/better-language-models/language_models_are_unsupervised_multitask_learners.pdf)

### Online Resources

1. **Hugging Face Transformers Documentation**
   - Comprehensive guide to using pre-trained models
   - [https://huggingface.co/docs/transformers](https://huggingface.co/docs/transformers)

2. **The Illustrated GPT-2** by Jay Alammar
   - Visual explanation of GPT-2 architecture
   - [https://jalammar.github.io/illustrated-gpt2/](https://jalammar.github.io/illustrated-gpt2/)

3. **Andrej Karpathy's nanoGPT**
   - Minimal, educational GPT implementation
   - [https://github.com/karpathy/nanoGPT](https://github.com/karpathy/nanoGPT)

4. **GPT in 60 Lines of NumPy** by Jay Mody
   - Ultra-minimal implementation for understanding
   - [https://jaykmody.com/blog/gpt-from-scratch/](https://jaykmody.com/blog/gpt-from-scratch/)

5. **Stanford CS224N: NLP with Deep Learning**
   - Lecture videos and notes on transformers
   - [https://web.stanford.edu/class/cs224n/](https://web.stanford.edu/class/cs224n/)

### Books

1. **"Natural Language Processing with Transformers"** by Tunstall, von Werra, and Wolf
   - Practical guide to transformer models
   - Published by O'Reilly Media, 2022

2. **"Speech and Language Processing"** by Jurafsky and Martin (3rd ed.)
   - Chapter on neural language models and transformers
   - [Free online draft](https://web.stanford.edu/~jurafsky/slp3/)

### Practical Tutorials

1. **Hugging Face Course**
   - Free course on using transformer models
   - [https://huggingface.co/course](https://huggingface.co/course)

2. **PyTorch Transformer Tutorial**
   - Official PyTorch guide to implementing transformers
   - [https://pytorch.org/tutorials/beginner/transformer_tutorial.html](https://pytorch.org/tutorials/beginner/transformer_tutorial.html)

### Datasets for Practice

1. **WikiText**: Language modeling dataset
2. **OpenWebText**: Open-source recreation of WebText
3. **The Pile**: Large, diverse text dataset
4. **C4 (Colossal Clean Crawled Corpus)**: Web text dataset

---

**Congratulations on completing Day 84!** 🎉

You now understand one of the most influential architectures in modern AI. GPT and its successors have transformed natural language processing and continue to push the boundaries of what's possible with language models.

Keep experimenting with the code, try fine-tuning models on your own datasets, and explore the latest developments in transformer architectures. The field is evolving rapidly, and understanding these fundamentals will help you stay at the forefront of AI research and applications.

See you in Lesson 85! 🚀
